In [76]:
import tensorflow as tf
import keras
import numpy as np
from keras.layers import *
from keras.models import *
from numpy import newaxis
from keras import backend as K

In [12]:
m1 = np.array([[1,5], [2,2]])
m2 = np.array([[2,6], [3,3]])
m1.shape

(2, 2)

In [83]:
# inputs
i1 = np.array([1,5], dtype=np.float32)
i2 = np.array([2,6], dtype=np.float32)
i3 = np.array([3,7], dtype=np.float32)
i4 = np.array([4,8], dtype=np.float32)
i5 = np.array([4,9], dtype=np.float32)
i1.shape

(2,)

In [82]:
i5.dtype


dtype('int32')

In [56]:
np.stack(m1).shape
m2[newaxis, :,:].shape
print("x(+)", x.predict([m1[newaxis, :,:], m2[newaxis, :,:]]))

(1, 2, 2)

In [84]:
# first network (simple addition of value)
xA = Input((1,))
xB = Input((1,))
xOut = keras.layers.Add()([xA,xB]) 
x = Model([xA, xB], xOut)

x.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_64 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_65 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
add_16 (Add)                     (None, 1)             0           input_64[0][0]                   
                                                                   input_65[0][0]                   
Total params: 0
Trainable params: 0
Non-trainable params: 0
____________________________________________________________________________________________________


In [85]:
print("x(+)", x.predict([i2, i3]))

x(+) [[  5.]
 [ 13.]]


In [86]:
# second network (simple multiplicaiton of values)
yA = Input((1,))
yB = Input((1,))
yOut = keras.layers.Multiply()([yA,yB])
y = Model([yA, yB], yOut)
print(y.summary())
print("y(*)", y.predict([i4, i5]))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_66 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_67 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
multiply_3 (Multiply)            (None, 1)             0           input_66[0][0]                   
                                                                   input_67[0][0]                   
Total params: 0
Trainable params: 0
Non-trainable params: 0
____________________________________________________________________________________________________
None
y(*) [[ 16.]
 [ 72.]]


In [87]:
print("x->y", y.predict([x.predict([i2, i3]),i3]))

x->y [[ 15.]
 [ 91.]]


In [108]:
K.set_learning_phase(1)
# same as second, with a 'static' value
zA = Input((1,))
zB = Input((1,))
zOut = Lambda(lambda z: z[0] * z[1][0])([zA, zB]) # multiply by a static value in the input
z = Model([zA, zB], zOut)

print(z.summary())

print("x->z", z.predict([x.predict([i2, i3]),i3]))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_108 (InputLayer)           (None, 1)             0                                            
____________________________________________________________________________________________________
input_109 (InputLayer)           (None, 1)             0                                            
____________________________________________________________________________________________________
lambda_17 (Lambda)               (None, 1)             0           input_108[0][0]                  
                                                                   input_109[0][0]                  
Total params: 0
Trainable params: 0
Non-trainable params: 0
____________________________________________________________________________________________________
None
x->z [[ 15.]
 [ 39.]]


In [89]:
# create new chained models with the same order
xyA = Input((1,))
xyB = Input((1,))
xy = Model([xyA, xyB], y([x([xyA, xyB]), xyB]))
print(xy.summary())
print("xy", xy.predict([i2, i3]))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_70 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_71 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
model_32 (Model)                 (None, 1)             0           input_70[0][0]                   
                                                                   input_71[0][0]                   
____________________________________________________________________________________________________
model_33 (Model)                 (None, 1)             0           model_32[1][0]          

In [109]:
xzA = Input((1,))
xzB = Input((1,))
print(xy.summary())
xz = Model([xzA, xzB], z([x([xzA, xzB]), xzB]))
print("xz", xz.predict([i2, i3])) # <-- complains about missing input

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_70 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_71 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
model_32 (Model)                 (None, 1)             0           input_70[0][0]                   
                                                                   input_71[0][0]                   
____________________________________________________________________________________________________
model_33 (Model)                 (None, 1)             0           model_32[1][0]          